# Shor's Algorithm
Shor's algorithm basically comes down to period-finding. Say you have a periodic function:

\begin{equation}
f(x) = a^x mod N
\end{equation}

where $a, N$ are positive integers with $a<N$ and they have no common factors. The period $r$ is the smallest non-zero integer such that:

\begin{equation}
a^r mod N = 1
\end{equation}

## How do we get from a period $r$ to factorisation?
So say we want to factor the number $N$ into it's prime factors. The procedure is the following:

1. Choose a random number $a$ between 1 and $N-1$.

2. Check that $a$ isn't already a non-trivial factor of $N$.

3. 